In [1]:
label_path = []
label_name= []

In [3]:
import sys
import os
import pandas as pd
import cv2
import pickle
import EDA_utils as EU
import json

# query 실행
query1 = (f'''CREATE TABLE IF NOT EXISTS TeyeD_GazeinTheWild_info (
        id INT(11) NOT NULL AUTO_INCREMENT primary key,
        img_dir VARCHAR(100) NOT NULL,
        img_format VARCHAR(10) NOT NULL,
        img_width INT(11) NOT NULL,
        img_height INT(11) NOT NULL,
        color_space VARCHAR(10) NOT NULL,
        label_dir VARCHAR(100) NOT NULL,
        label_format VARCHAR(10) NOT NULL,
        camera_3d_center VARCHAR(200) NOT NULL,
        camera_distance VARCHAR(200) NOT NULL,
        iris_loc VARCHAR(200) NOT NULL,
        eye_loc VARCHAR(200) NOT NULL,
        gaze_angle_az VARCHAR(200) NOT NULL,
        gaze_angle_el VARCHAR(200) NOT NULL,
        ortho_scale VARCHAR(200) NOT NULL,
        cam_az VARCHAR(200) NOT NULL,
        cam_el VARCHAR(200) NOT NULL,
        glasses VARCHAR(200) NOT NULL,
        pupil VARCHAR(200) NOT NULL,
        eye_lid VARCHAR(200) NOT NULL,
        iris_rot VARCHAR(200) NOT NULL,
        left_corner VARCHAR(200) NOT NULL,
        right_corner VARCHAR(200) NOT NULL,
        sclera VARCHAR(200) NOT NULL)''')
try:
    EU.MakeSQL.create_table(query1)
except Exception as e:
    print(f"Error during table creation: {e}")

query2 = (f'''INSERT INTO TeyeD_GazeinTheWild_info 
        (img_dir, img_format, img_width, img_height, color_space,
          label_dir, label_format, camera_3d_center, camera_distance,
          iris_loc, eye_loc, gaze_angle_az, gaze_angle_el, ortho_scale, cam_az, cam_el,
          glasses, pupil, eye_lid, iris_rot, left_corner, right_corner, sclera) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)''')

# label 경로
label_path = 'Z:\\RIT_Eyes_s_general\\data\\IR\\labeldata'
image_path = 'Z:\\RIT_Eyes_s_general\\data\\IR\\rawdata'

value_list1 = []
error_list = []
count = 0

for data_name in os.listdir(image_path):
    image_path1 = os.path.join(image_path,data_name) 
    for accesory in os.listdir(image_path1):
        image_path2 = os.path.join(image_path1,accesory)
        for image_name in os.listdir(image_path2):
            image = os.path.join(image_path2,image_name)
            # print(image_path1)
            # print(image_path2)
            # print(image)
            label_path = image_path1.replace("rawdata","labeldata")
            label_name = os.listdir(label_path)
            # print(label_name)
            # Check if the list is empty
            if not label_name:
                print(f"No files found in {label_path}")
                continue  # Skip to the next iteration of the loop

            label = os.path.join(label_path,str(label_name[0]).replace('[','').replace(']','').replace("'",'').replace("'",''))
            try:
                with open(label, 'rb') as f:
                    text = pickle.load(f)
                    img_dir = image
                    img_format = 'tif'
                    image = cv2.imread(img_dir)
                    img_width = image.shape[1]
                    img_height = image.shape[0]
                    color_space = 'IR'
                    label_dir = label
                    label_format = 'p'
                    camera_3d_center = str(text["camera_3d_center"][count])
                    camera_distance = str(text["camera_distance"][count])
                    iris_loc = str(text["iris_loc"][count])
                    eye_loc = str(text["eye_loc"][count])
                    gaze_angle_az = str(text["gaze_angle_az"][count])
                    gaze_angle_el = str(text["gaze_angle_el"][count])
                    ortho_scale = str(text["ortho_scale"][count])
                    cam_az = str(text["cam_az"][count])
                    cam_el = str(text["cam_el"][count])
                    glasses = str(text["glasses"][count])
                    pupil = str(text["pupil"][count])
                    eye_lid = str(text["eye_lid"][count])
                    iris_rot = str(text["iris_rot"][count])
                    left_corner = str(text["left_corner"][count])
                    right_corner = str(text["right_corner"][count])
                    sclera = str(text["sclera"][count])

                    img_dir = img_dir.replace('Z:', 'DataBase')
                    label_dir = label_dir.replace('Z:', 'DataBase')

                    value = (img_dir, img_format, img_width, img_height, color_space, label_dir, label_format, camera_3d_center, camera_distance, iris_loc, eye_loc, gaze_angle_az, gaze_angle_el, ortho_scale, cam_az, cam_el,
                    glasses, pupil, eye_lid, iris_rot, left_corner, right_corner, sclera)
                    value_list1.append(value)

                    count += 1

            except Exception as ex:
                error_list.append((label, str(ex)))
                print(f"Error loading data from file: {label}")
                print(f"Error details: {ex}")

                # query 실행
            if len(value_list1) >= 300:
                # print(value_list1[0])
                EU.MakeSQL.insert_dataset_values(query2, value_list1)
                value_list1 = []
                print(f'{len(value_list1)} / {len(label_name)}')

        count = 0

EU.MakeSQL.insert_dataset_values(query2, value_list1)
# print(value_list2)
value_list1 = []
print('done')

Error during table creation: 1050: Table 'RIT_Eyes_s_general_info' already exists
('DataBase\\RIT_Eyes_s_general\\data\\IR\\rawdata\\1\\mask-withoutskin\\0000.tif', 'tif', 640, 480, 'IR', 'DataBase\\RIT_Eyes_s_general\\data\\IR\\labeldata\\1\\1-general.p', 'p', '[ 0.47760594 -4.44124651  0.18239957]', '3.2580194059927954', '[0.03500239923596382, -1.0023376941680908, 0.010153090581297874]', '[0.0, 0.0, 0.0]', '1.9999997710638548', '0.5799966692718073', '3.25801944732666', '189.28659847399365', '-4.293334780411306', '0', '0.7569016218185425', '0.0', '[-1.5707963705062866, 0.0, 1.046067476272583]', '[1.1374797821044922, -0.36298900842666626, -0.11673785746097565]', '[-0.7808918356895447, -0.9031707048416138, -0.17035934329032898]', '[0.0, 0.0, 6.098071575164795]')
insert 300 done
0 / 1
('DataBase\\RIT_Eyes_s_general\\data\\IR\\rawdata\\1\\mask-withoutskin\\0300.tif', 'tif', 640, 480, 'IR', 'DataBase\\RIT_Eyes_s_general\\data\\IR\\labeldata\\1\\1-general.p', 'p', '[ 2.83758259 -4.6311388  

In [5]:
# query 실행
query1 = (f'''CREATE TABLE IF NOT EXISTS RIT_Eyes_s_natural_info (
        id INT(11) NOT NULL AUTO_INCREMENT primary key,
        img_dir VARCHAR(100) NOT NULL,
        img_format VARCHAR(10) NOT NULL,
        img_width INT(11) NOT NULL,
        img_height INT(11) NOT NULL,
        color_space VARCHAR(10) NOT NULL,
        label_dir VARCHAR(100) NOT NULL,
        label_format VARCHAR(10) NOT NULL,
        camera_3d_center VARCHAR(200) NOT NULL,
        camera_distance VARCHAR(200) NOT NULL,
        iris_loc VARCHAR(200) NOT NULL,
        eye_loc VARCHAR(200) NOT NULL,
        gaze_angle_az VARCHAR(200) NOT NULL,
        gaze_angle_el VARCHAR(200) NOT NULL,
        ortho_scale VARCHAR(200) NOT NULL,
        cam_az VARCHAR(200) NOT NULL,
        cam_el VARCHAR(200) NOT NULL,
        glasses VARCHAR(200) NOT NULL,
        pupil VARCHAR(200) NOT NULL,
        eye_lid VARCHAR(200) NOT NULL,
        iris_rot VARCHAR(200) NOT NULL,
        left_corner VARCHAR(200) NOT NULL,
        right_corner VARCHAR(200) NOT NULL,
        sclera VARCHAR(200) NOT NULL)''')
try:
    EU.MakeSQL.create_table(query1)
except Exception as e:
    print(f"Error during table creation: {e}")

query2 = (f'''INSERT INTO RIT_Eyes_s_natural_info 
        (img_dir, img_format, img_width, img_height, color_space,
          label_dir, label_format, camera_3d_center, camera_distance,
          iris_loc, eye_loc, gaze_angle_az, gaze_angle_el, ortho_scale, cam_az, cam_el,
          glasses, pupil, eye_lid, iris_rot, left_corner, right_corner, sclera) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)''')

# label 경로
label_path = 'Z:\\RIT_Eyes_s_natural\\data\\RGB\\labeldata'
image_path = 'Z:\\RIT_Eyes_s_natural\\data\\RGB\\rawdata'

value_list1 = []
error_list = []
count = 0

for data_name in os.listdir(image_path):
    image_path1 = os.path.join(image_path,data_name) 
    for accesory in os.listdir(image_path1):
        image_path2 = os.path.join(image_path1,accesory)
        for image_name in os.listdir(image_path2):
            image = os.path.join(image_path2,image_name)
            # print(image_path1)
            # print(image_path2)
            # print(image)
            label_path = image_path1.replace("rawdata","labeldata")
            label_name = os.listdir(label_path)
            # print(label_name)
            # Check if the list is empty
            if not label_name:
                print(f"No files found in {label_path}")
                continue  # Skip to the next iteration of the loop

            label = os.path.join(label_path,str(label_name[0]).replace('[','').replace(']','').replace("'",'').replace("'",''))
            try:
                with open(label, 'rb') as f:
                    text = pickle.load(f)
                    img_dir = image
                    img_format = 'tif'
                    image = cv2.imread(img_dir)
                    img_width = image.shape[1]
                    img_height = image.shape[0]
                    color_space = 'RGB'
                    label_dir = label
                    label_format = 'p'
                    camera_3d_center = str(text["camera_3d_center"][count])
                    camera_distance = str(text["camera_distance"][count])
                    iris_loc = str(text["iris_loc"][count])
                    eye_loc = str(text["eye_loc"][count])
                    gaze_angle_az = str(text["gaze_angle_az"][count])
                    gaze_angle_el = str(text["gaze_angle_el"][count])
                    ortho_scale = str(text["ortho_scale"][count])
                    cam_az = str(text["cam_az"][count])
                    cam_el = str(text["cam_el"][count])
                    glasses = str(text["glasses"][count])
                    pupil = str(text["pupil"][count])
                    eye_lid = str(text["eye_lid"][count])
                    iris_rot = str(text["iris_rot"][count])
                    left_corner = str(text["left_corner"][count])
                    right_corner = str(text["right_corner"][count])
                    sclera = str(text["sclera"][count])

                    img_dir = img_dir.replace('Z:', 'DataBase')
                    label_dir = label_dir.replace('Z:', 'DataBase')

                    value = (img_dir, img_format, img_width, img_height, color_space, label_dir, label_format, camera_3d_center, camera_distance, iris_loc, eye_loc, gaze_angle_az, gaze_angle_el, ortho_scale, cam_az, cam_el,
                    glasses, pupil, eye_lid, iris_rot, left_corner, right_corner, sclera)
                    value_list1.append(value)

                    count += 1

            except Exception as ex:
                error_list.append((label, str(ex)))
                print(f"Error loading data from file: {label}")
                print(f"Error details: {ex}")

                # query 실행
            if len(value_list1) >= 300:
                # print(value_list1[0])
                EU.MakeSQL.insert_dataset_values(query2, value_list1)
                value_list1 = []
                print(f'{len(value_list1)} / {len(label_name)}')

        count = 0

EU.MakeSQL.insert_dataset_values(query2, value_list1)
# print(value_list2)
value_list1 = []
print('done')

Error during table creation: 1050: Table 'RIT_Eyes_s_natural_info' already exists
('DataBase\\RIT_Eyes_s_natural\\data\\RGB\\rawdata\\1\\mask-withoutskin\\0000.tif', 'tif', 640, 480, 'RGB', 'DataBase\\RIT_Eyes_s_natural\\data\\RGB\\labeldata\\1\\1-natural.p', 'p', '[ 0.47760594 -4.44124651  0.18239957]', '3.2580194059927954', '[0.03500239923596382, -1.0023376941680908, 0.010153090581297874]', '[0.0, 0.0, 0.0]', '1.9999997710638548', '0.5799966692718073', '3.25801944732666', '189.28659847399365', '-4.293334780411306', '0', '0.7569016218185425', '0.0', '[-1.5707963705062866, 0.0, 1.046067476272583]', '[1.1374797821044922, -0.36298900842666626, -0.11673785746097565]', '[-0.7808918356895447, -0.9031707048416138, -0.17035934329032898]', '[0.0, 0.0, 6.098071575164795]')
insert 300 done
0 / 1
('DataBase\\RIT_Eyes_s_natural\\data\\RGB\\rawdata\\1\\mask-withoutskin\\0300.tif', 'tif', 640, 480, 'RGB', 'DataBase\\RIT_Eyes_s_natural\\data\\RGB\\labeldata\\1\\1-natural.p', 'p', '[ 2.83758259 -4.631

In [2]:
import sys
import os
import pandas as pd
import cv2
import pickle
import EDA_utils as EU
import json

# query 실행
query1 = (f'''CREATE TABLE IF NOT EXISTS RIT_Eyes_s_nvgaze_info (
        id INT(11) NOT NULL AUTO_INCREMENT primary key,
        img_dir VARCHAR(100) NOT NULL,
        img_format VARCHAR(10) NOT NULL,
        img_width INT(11) NOT NULL,
        img_height INT(11) NOT NULL,
        color_space VARCHAR(10) NOT NULL,
        label_dir VARCHAR(100) NOT NULL,
        label_format VARCHAR(10) NOT NULL,
        camera_3d_center VARCHAR(200) NOT NULL,
        camera_distance VARCHAR(200) NOT NULL,
        iris_loc VARCHAR(200) NOT NULL,
        eye_loc VARCHAR(200) NOT NULL,
        gaze_angle_az VARCHAR(200) NOT NULL,
        gaze_angle_el VARCHAR(200) NOT NULL,
        ortho_scale VARCHAR(200) NOT NULL,
        cam_az VARCHAR(200) NOT NULL,
        cam_el VARCHAR(200) NOT NULL,
        glasses VARCHAR(200) NOT NULL,
        pupil VARCHAR(200) NOT NULL,
        eye_lid VARCHAR(200) NOT NULL,
        iris_rot VARCHAR(200) NOT NULL,
        left_corner VARCHAR(200) NOT NULL,
        right_corner VARCHAR(200) NOT NULL,
        sclera VARCHAR(200) NOT NULL)''')
try:
    EU.MakeSQL.create_table(query1)
except Exception as e:
    print(f"Error during table creation: {e}")

query2 = (f'''INSERT INTO RIT_Eyes_s_nvgaze_info 
        (img_dir, img_format, img_width, img_height, color_space,
          label_dir, label_format, camera_3d_center, camera_distance,
          iris_loc, eye_loc, gaze_angle_az, gaze_angle_el, ortho_scale, cam_az, cam_el,
          glasses, pupil, eye_lid, iris_rot, left_corner, right_corner, sclera) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)''')

# label 경로
label_path = 'Z:\\RIT_Eyes_s_nvgaze\\data\\IR\\labeldata'
image_path = 'Z:\\RIT_Eyes_s_nvgaze\\data\\IR\\rawdata'

value_list1 = []
error_list = []
count = 0

for data_name in os.listdir(image_path):
    image_path1 = os.path.join(image_path,data_name) 
    for accesory in os.listdir(image_path1):
        image_path2 = os.path.join(image_path1,accesory)
        for image_name in os.listdir(image_path2):
            image = os.path.join(image_path2,image_name)
            # print(image_path1)
            # print(image_path2)
            # print(image)
            label_path = image_path1.replace("rawdata","labeldata")
            label_name = os.listdir(label_path)
            # print(label_name)
            # Check if the list is empty
            if not label_name:
                print(f"No files found in {label_path}")
                continue  # Skip to the next iteration of the loop

            label = os.path.join(label_path,str(label_name[0]).replace('[','').replace(']','').replace("'",'').replace("'",''))
            try:
                with open(label, 'rb') as f:
                    text = pickle.load(f)
                    img_dir = image
                    img_format = 'tif'
                    image = cv2.imread(img_dir)
                    img_width = image.shape[1]
                    img_height = image.shape[0]
                    color_space = 'IR'
                    label_dir = label
                    label_format = 'p'
                    camera_3d_center = str(text["camera_3d_center"][count])
                    camera_distance = str(text["camera_distance"][count])
                    iris_loc = str(text["iris_loc"][count])
                    eye_loc = str(text["eye_loc"][count])
                    gaze_angle_az = str(text["gaze_angle_az"][count])
                    gaze_angle_el = str(text["gaze_angle_el"][count])
                    ortho_scale = str(text["ortho_scale"][count])
                    cam_az = str(text["cam_az"][count])
                    cam_el = str(text["cam_el"][count])
                    glasses = str(text["glasses"][count])
                    pupil = str(text["pupil"][count])
                    eye_lid = str(text["eye_lid"][count])
                    iris_rot = str(text["iris_rot"][count])
                    left_corner = str(text["left_corner"][count])
                    right_corner = str(text["right_corner"][count])
                    sclera = str(text["sclera"][count])

                    img_dir = img_dir.replace('Z:', 'DataBase')
                    label_dir = label_dir.replace('Z:', 'DataBase')

                    value = (img_dir, img_format, img_width, img_height, color_space, label_dir, label_format, camera_3d_center, camera_distance, iris_loc, eye_loc, gaze_angle_az, gaze_angle_el, ortho_scale, cam_az, cam_el,
                    glasses, pupil, eye_lid, iris_rot, left_corner, right_corner, sclera)
                    value_list1.append(value)

                    count += 1

            except Exception as ex:
                error_list.append((label, str(ex)))
                print(f"Error loading data from file: {label}")
                print(f"Error details: {ex}")

                # query 실행
            if len(value_list1) >= 300:
                print(value_list1[0])
                EU.MakeSQL.insert_dataset_values(query2, value_list1)
                value_list1 = []
                print(f'{len(value_list1)} / {len(label_name)}')

        count = 0

EU.MakeSQL.insert_dataset_values(query2, value_list1)
# print(value_list2)
value_list1 = []
print('done')

Error during table creation: 1050: Table 'RIT_Eyes_s_nvgaze_info' already exists
('DataBase\\RIT_Eyes_s_nvgaze\\data\\IR\\rawdata\\1\\mask-withoutskin\\0000.tif', 'tif', 640, 480, 'IR', 'DataBase\\RIT_Eyes_s_nvgaze\\data\\IR\\labeldata\\1\\1-snvgaze.p', 'p', '[-0.017064335517883306, -5.02645169052124, -0.08398039662551879]', '3.8038271432595505', '[0.03500239923596382, -1.0023376941680908, 0.010153090581297874]', '[0.0, 0.0, 0.0]', '1.9999997710638548', '0.5799966692718073', '3.8028616905212402', '0', '90', '0', '0.7630667036738225', '0.0', '[-1.5707963705062866, 0.0, 1.046067476272583]', '[1.1374797821044922, -0.36298900842666626, -0.11673785746097565]', '[-0.7808918356895447, -0.9031707048416138, -0.17035934329032898]', '[0.0, 0.0, 6.098071575164795]')
insert 300 done
0 / 1
('DataBase\\RIT_Eyes_s_nvgaze\\data\\IR\\rawdata\\1\\mask-withoutskin\\0300.tif', 'tif', 640, 480, 'IR', 'DataBase\\RIT_Eyes_s_nvgaze\\data\\IR\\labeldata\\1\\1-snvgaze.p', 'p', '[-0.019101935200691228, -5.6357852

In [2]:
# query 실행
query1 = (f'''CREATE TABLE IF NOT EXISTS RIT_Eyes_s_openeds_info (
        
        id INT(11) NOT NULL AUTO_INCREMENT primary key,
        img_dir VARCHAR(100) NOT NULL,
        img_format VARCHAR(10) NOT NULL,
        img_width INT(11) NOT NULL,
        img_height INT(11) NOT NULL,
        color_space VARCHAR(10) NOT NULL,
        label_dir VARCHAR(100) NOT NULL,
        label_format VARCHAR(10) NOT NULL,
        camera_3d_center VARCHAR(200) NOT NULL,
        camera_distance VARCHAR(200) NOT NULL,
        iris_loc VARCHAR(200) NOT NULL,
        eye_loc VARCHAR(200) NOT NULL,
        gaze_angle_az VARCHAR(200) NOT NULL,
        gaze_angle_el VARCHAR(200) NOT NULL,
        ortho_scale VARCHAR(200) NOT NULL,
        cam_az VARCHAR(200) NOT NULL,
        cam_el VARCHAR(200) NOT NULL,
        glasses VARCHAR(200) NOT NULL,
        pupil VARCHAR(200) NOT NULL,
        eye_lid VARCHAR(200) NOT NULL,
        iris_rot VARCHAR(200) NOT NULL,
        left_corner VARCHAR(200) NOT NULL,
        right_corner VARCHAR(200) NOT NULL,
        sclera VARCHAR(200) NOT NULL)''')
try:
    EU.MakeSQL.create_table(query1)
except Exception as e:
    print(f"Error during table creation: {e}")

query2 = (f'''INSERT INTO RIT_Eyes_s_openeds_info 
        (img_dir, img_format, img_width, img_height, color_space,
          label_dir, label_format, camera_3d_center, camera_distance,
          iris_loc, eye_loc, gaze_angle_az, gaze_angle_el, ortho_scale, cam_az, cam_el,
          glasses, pupil, eye_lid, iris_rot, left_corner, right_corner, sclera) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)''')

# label 경로
label_path = 'Z:\\RIT_Eyes_s_openeds\\data\\IR\\labeldata'
image_path = 'Z:\\RIT_Eyes_s_openeds\\data\\IR\\rawdata'

value_list1 = []
error_list = []
count = 0

for data_name in os.listdir(image_path):
    image_path1 = os.path.join(image_path,data_name) 
    for accesory in os.listdir(image_path1):
        image_path2 = os.path.join(image_path1,accesory)
        for image_name in os.listdir(image_path2):
            image = os.path.join(image_path2,image_name)
            # print(image_path1)
            # print(image_path2)
            # print(image)
            label_path = image_path1.replace("rawdata","labeldata")
            label_name = os.listdir(label_path)
            # print(label_name)
            # Check if the list is empty
            if not label_name:
                print(f"No files found in {label_path}")
                continue  # Skip to the next iteration of the loop

            label = os.path.join(label_path,str(label_name[0]).replace('[','').replace(']','').replace("'",'').replace("'",''))
            try:
                with open(label, 'rb') as f:
                    text = pickle.load(f)
                    img_dir = image
                    img_format = 'tif'
                    image = cv2.imread(img_dir)
                    img_width = image.shape[1]
                    img_height = image.shape[0]
                    color_space = 'IR'
                    label_dir = label
                    label_format = 'p'
                    camera_3d_center = str(text["camera_3d_center"][count])
                    camera_distance = str(text["camera_distance"][count])
                    iris_loc = str(text["iris_loc"][count])
                    eye_loc = str(text["eye_loc"][count])
                    gaze_angle_az = str(text["gaze_angle_az"][count])
                    gaze_angle_el = str(text["gaze_angle_el"][count])
                    ortho_scale = str(text["ortho_scale"][count])
                    cam_az = str(text["cam_az"][count])
                    cam_el = str(text["cam_el"][count])
                    glasses = str(text["glasses"][count])
                    pupil = str(text["pupil"][count])
                    eye_lid = str(text["eye_lid"][count])
                    iris_rot = str(text["iris_rot"][count])
                    left_corner = str(text["left_corner"][count])
                    right_corner = str(text["right_corner"][count])
                    sclera = str(text["sclera"][count])

                    img_dir = img_dir.replace('Z:', 'DataBase')
                    label_dir = label_dir.replace('Z:', 'DataBase')

                    value = (img_dir, img_format, img_width, img_height, color_space, label_dir, label_format, camera_3d_center, camera_distance, iris_loc, eye_loc, gaze_angle_az, gaze_angle_el, ortho_scale, cam_az, cam_el,
                    glasses, pupil, eye_lid, iris_rot, left_corner, right_corner, sclera)
                    value_list1.append(value)

                    count += 1

            except Exception as ex:
                error_list.append((label, str(ex)))
                print(f"Error loading data from file: {label}")
                print(f"Error details: {ex}")

                # query 실행
            if len(value_list1) >= 300:
                # print(value_list1[0])
                EU.MakeSQL.insert_dataset_values(query2, value_list1)
                value_list1 = []
                print(f'{len(value_list1)} / {len(label_name)}')

        count = 0

EU.MakeSQL.insert_dataset_values(query2, value_list1)
# print(value_list2)
value_list1 = []
print('done')

create table done
('DataBase\\RIT_Eyes_s_openeds\\data\\IR\\rawdata\\1\\mask-withoutskin\\0000.tif', 'tif', 400, 640, 'IR', 'DataBase\\RIT_Eyes_s_openeds\\data\\IR\\labeldata\\1\\1-sopeneds.p', 'p', '[0.20391331613063812, -5.499320974873645, -1.2734660813419565]', '4.466001776792948', '[0.03500239923596382, -1.0023376941680908, 0.010153090581297874]', '[0.0, 0.0, 0.0]', '1.9999997710638548', '0.5799966692718073', '4.341691017150879', '0', '100', '0', '0.7630667036738225', '0.0', '[-1.5707963705062866, 0.0, 1.046067476272583]', '[1.1374797821044922, -0.36298900842666626, -0.11673785746097565]', '[-0.7808918356895447, -0.9031707048416138, -0.17035934329032898]', '[0.0, 0.0, 6.098071575164795]')
insert 300 done
0 / 1
('DataBase\\RIT_Eyes_s_openeds\\data\\IR\\rawdata\\1\\mask-withoutskin\\0300.tif', 'tif', 400, 640, 'IR', 'DataBase\\RIT_Eyes_s_openeds\\data\\IR\\labeldata\\1\\1-sopeneds.p', 'p', '[-0.41889479756355286, -5.295591838796564, -1.0232833346000054]', '4.219452643348248', '[-0.25